In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
from datetime import date
import pickle
import requests
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import tqdm
import sys
sys.path.append("../")
import src.soporte as sp

c:\Users\Lin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv("../data/datamarket_productos_de_supermercados.csv")
df.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,FLEX champú con Keratina para cabello normal f...,NaN,3.39,5.22,l,2021-03-06 00:00:00,782a516202a4c7d4977341602ed10173
1,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,FLEX champú con Keratina fortificante frasco 6...,NaN,3.39,5.22,l,2021-03-06 00:00:00,fcadc0dcbdfaa1dc2aa6c99523c3fa47
2,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,GLISS champú reparador total frasco 250 ml,NaN,2.99,11.96,l,2021-03-06 00:00:00,bf86b25bafae2e646edba24ed9a0af7c
3,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,GLISS champú reparador ultimate frasco 250 ml,NaN,2.99,11.96,l,2021-03-06 00:00:00,df3b30f9e481ec851d606a99fd8ad8b7
4,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú protección color cabellos c...,NaN,3.99,13.30,l,2021-03-06 00:00:00,9c94c0754747ab0d37067ede61a682fc


In [3]:
# Cambiamos los precios para que sean más uniformes las unidades de medida
df["reference_price"] = df.apply(lambda x: sp.limpiar_reference(x["reference_price"], x["reference_unit"]), axis = 1)

In [4]:
# Cambio las unidades después
df["reference_unit"] = df["reference_unit"].apply(lambda x: sp.limpiar_runit(x ))


In [5]:
df["reference_unit"].value_counts()

kg        3817199
l         2529459
ud        1416919
lavado      61274
m           13107
Name: reference_unit, dtype: int64

In [6]:
# No me sirve description y hay muchos nulos, las url muchas no funcionan, el product_id lo voy a generar yo mismo
df.drop(["description", "url", "product_id"], axis = 1, inplace = True)

In [7]:
# Las horas estan vacías no sirven
df["insert_date"] = pd.to_datetime(df["insert_date"])

In [8]:
# split por supermercados para limpiar más fino
dfm = df[df["supermarket"] == "mercadona-es"]
dfc = df[df["supermarket"] == "carrefour-es"]
dfd = df[df["supermarket"] == "dia-es"]

In [9]:
# Primero DIA
# Hay una categoria rara, la drop
dfd2 = dfd[dfd["category"] != "soy_solidario_soy_solidario_soy_solidario"]

In [10]:
# Limpiamos las categorias empezamos creando subcategorias BIEN
dfd2["subcategoria"] = dfd2.apply(lambda x: sp.dia_subcat(x["category"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_40800\2621852338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd2["subcategoria"] = dfd2.apply(lambda x: sp.dia_subcat(x["category"]), axis = 1)


In [11]:
# A partir de las subcategorias BIEN ahora creo las categorias grandes BIEN
dfd2["category"] = dfd2.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_40800\593860639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd2["category"] = dfd2.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)


In [12]:
# Limpiamos las categorias de Mercadona
# Limpiamos las categorias empezamos creando subcategorias BIEN
dfm["subcategoria"] = dfm.apply(lambda x: sp.mer_subcat(x["category"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_40800\3216612271.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm["subcategoria"] = dfm.apply(lambda x: sp.mer_subcat(x["category"]), axis = 1)


In [13]:
# A partir de las subcategorias BIEN ahora creo las categorias grandes BIEN
dfm["categoria"] = dfm.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)


C:\Users\Lin\AppData\Local\Temp\ipykernel_40800\4076117406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm["categoria"] = dfm.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)


In [14]:
dfd.to_csv("../data/dia_limpio.csv")

In [15]:
dfm.to_csv("../data/mercadona_limpio.csv")